# 01. Molecular data exploration (NCBI + Bloom)

This notebook performs initial exploration and consistency checks of the molecular data used in Module 1:
- SARS-CoV-2 genomes and metadata (Delta and Omicron) obtained from NCBI Virus.
- Spike phenotypes at clade and mutation level obtained from the Bloom lab submodule (`SARS2-spike-predictor-phenos`).

In [ ]:
# Imports

import pandas as pd
from pathlib import Path

# Paths (adjust if the notebook is not at the project root)
BASE = Path("..")
DATA = BASE / "data"
EXTERNAL = BASE / "external" / "SARS2-spike-predictor-phenos" / "results"

## 1. Load NCBI molecular datasets (Delta & Omicron)

We load the cleaned metadata for Delta and Omicron and run basic checks:
- number of genomes
- column types and missing values
- distribution over collection dates and geographic locations.


In [12]:
# Load metadata
delta = pd.read_csv(DATA / "delta" / "delta_meta_clean.csv")
omicron = pd.read_csv(DATA / "omicron" / "omicron_meta_clean.csv")

print("Delta shape:", delta.shape)
print("Omicron shape:", omicron.shape)

Delta shape: (29677, 4)
Omicron shape: (118530, 4)


In [13]:
# Basic info

print("=== Delta info ===")
delta.info()
print("\n=== Omicron info ===")
omicron.info()

=== Delta info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29677 entries, 0 to 29676
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   accession                29677 non-null  object
 1   virus_name               29677 non-null  object
 2   geographic_location      29677 non-null  object
 3   isolate_collection_date  29677 non-null  object
dtypes: object(4)
memory usage: 927.5+ KB

=== Omicron info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118530 entries, 0 to 118529
Data columns (total 4 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   accession                118530 non-null  object
 1   virus_name               118530 non-null  object
 2   geographic_location      118530 non-null  object
 3   isolate_collection_date  118530 non-null  object
dtypes: object(4)
memory usage: 3.6+ MB


### 1.1. Convert collection dates and inspect ranges


In [14]:
# Convert dates to datetime
for df in (delta, omicron):
    df["isolate_collection_date"] = pd.to_datetime(
        df["isolate_collection_date"], errors="coerce"
    )

print("Delta date range:",
      delta["isolate_collection_date"].min(),
      "→",
      delta["isolate_collection_date"].max())

print("Omicron date range:",
      omicron["isolate_collection_date"].min(),
      "→",
      omicron["isolate_collection_date"].max())

Delta date range: 2020-07-07 00:00:00 → 2025-03-28 00:00:00
Omicron date range: 2020-09-15 00:00:00 → 2023-10-18 00:00:00


### 1.2. Geographic distribution

We inspect the most represented countries / regions to ensure geographic diversity.

In [15]:
print("Delta top 10 locations:")
print(delta["geographic_location"].value_counts().head(10))

print("\nOmicron  top 10 locations:")
print(omicron["geographic_location"].value_counts().head(10))

Delta top 10 locations:
geographic_location
United Kingdom:England                                          16152
USA: Florida                                                     1923
USA                                                              1170
France                                                           1083
United Kingdom:Wales                                              970
Iceland                                                           528
Germany:Northrhine Westphalia; Duesseldorf metropolitan area      497
USA: California                                                   410
United Kingdom:Northern_Ireland                                   409
Slovakia                                                          388
Name: count, dtype: int64

Omicron  top 10 locations:
geographic_location
United Kingdom:England             62942
United Kingdom:Wales               15286
USA                                 5661
USA: California                     5509
United Kingdom:Sco

## 2. Check mapping between metadata and FASTA sequences

We verify that the `accession` values in the metadata also appear in the FASTA headers (`genomic.fna`), ensuring alignment between tables and sequences.


In [16]:
from itertools import islice

def load_fasta_accessions(fasta_path, n=None):
    """Read FASTA headers and collect accession IDs (first token after '>')."""
    accs = set()
    with open(fasta_path) as fh:
        for line in fh:
            if line.startswith(">"):
                header = line[1:].strip()
                first_token = header.split()[0]
                accs.add(first_token)
            if n is not None and len(accs) >= n:
                break
    return accs

delta_fasta_accs = load_fasta_accessions(
    DATA / "delta" / "ncbi_dataset" / "data" / "genomic.fna", n=10000
)
omicron_fasta_accs = load_fasta_accessions(
    DATA / "omicron" / "ncbi_dataset" / "data" / "genomic.fna", n=10000
)

for df, fasta_accs, name in [
    (delta, delta_fasta_accs, "Delta"),
    (omicron, omicron_fasta_accs, "Omicron"),
]:
    some_acc = df["accession"].iloc[0]
    print(f"\n{name} example accession from metadata: {some_acc}")
    print("In FASTA?", some_acc in fasta_accs)


Delta example accession from metadata: PQ993075.1
In FASTA? True

Omicron example accession from metadata: PQ965319.1
In FASTA? True


## 3. Load external spike phenotypes (Bloom lab)

We load the clade-level and mutation-level spike phenotype tables from the `SARS2-spike-predictor-phenos` submodule and inspect their main columns.


In [18]:
clade_pheno = pd.read_csv(EXTERNAL / "clade_phenotypes.csv")
mut_pheno = pd.read_csv(EXTERNAL / "mutation_phenotypes.csv")

print("Clade phenotypes shape:", clade_pheno.shape)
print("Mutation phenotypes shape:", mut_pheno.shape)

print("\nClade phenotypes columns:")
print(clade_pheno.columns.tolist())

print("\nMutation phenotypes columns:")
print(mut_pheno.columns.tolist()[:10])

Clade phenotypes shape: (3913, 19)
Mutation phenotypes shape: (54294, 6)

Clade phenotypes columns:
['clade', 'date', 'parent', 'spike muts from Wuhan-Hu-1', 'number spike muts from Wuhan-Hu-1', 'spike muts from XBB.1.5', 'descendant of BA.2.86', 'descendant of XBB', 'descendant of BA.2', 'clade growth', 'clade growth HDI 95', 'spike pseudovirus DMS human sera escape relative to XBB.1.5', 'spike pseudovirus DMS ACE2 binding relative to XBB.1.5', 'spike pseudovirus DMS spike mediated entry relative to XBB.1.5', 'RBD yeast-display DMS ACE2 affinity relative to XBB.1.5', 'RBD yeast-display DMS RBD expression relative to XBB.1.5', 'RBD yeast-display DMS escape relative to XBB.1.5', 'EVEscape relative to XBB.1.5', 'Hamming distance relative to XBB.1.5']

Mutation phenotypes columns:
['ref_clade', 'phenotype', 'site', 'wildtype', 'mutant', 'mutation_effect']


### 3.1. Example: clades related to Delta and Omicron

We inspect a few rows in `clade_phenotypes` for clades related to Delta and Omicron to see what “growth” and phenotype values are available for validation.

In [19]:
# Adjust filters depending on actual clade naming
delta_like = clade_pheno[clade_pheno["clade"].str.contains("B.1.617.2", na=False)]
omicron_like = clade_pheno[clade_pheno["clade"].str.contains("BA.1", na=False)]

print("Delta-like clades (example):")
display(delta_like.head())

print("\nOmicron-like clades (example):")
display(omicron_like.head())

Delta-like clades (example):


,clade,date,parent,spike muts from Wuhan-Hu-1,number spike muts from Wuhan-Hu-1,spike muts from XBB.1.5,descendant of BA.2.86,descendant of XBB,descendant of BA.2,clade growth,clade growth HDI 95,spike pseudovirus DMS human sera escape relative to XBB.1.5,spike pseudovirus DMS ACE2 binding relative to XBB.1.5,spike pseudovirus DMS spike mediated entry relative to XBB.1.5,RBD yeast-display DMS ACE2 affinity relative to XBB.1.5,RBD yeast-display DMS RBD expression relative to XBB.1.5,RBD yeast-display DMS escape relative to XBB.1.5,EVEscape relative to XBB.1.5,Hamming distance relative to XBB.1.5
2444,B.1.617.2,2021-04-21,B.1.617,T19R G142D R158G L452R T478K D614G P681R D950N...,10,I19R -24L -25P -26P S27A A83V -144Y Q146H E156...,False,False,False,NaN,NaN,-5.368,4.596,-7.942,-4.48,-0.22,1.221,63.39,44



Omicron-like clades (example):


,clade,date,parent,spike muts from Wuhan-Hu-1,number spike muts from Wuhan-Hu-1,spike muts from XBB.1.5,descendant of BA.2.86,descendant of XBB,descendant of BA.2,clade growth,clade growth HDI 95,spike pseudovirus DMS human sera escape relative to XBB.1.5,spike pseudovirus DMS ACE2 binding relative to XBB.1.5,spike pseudovirus DMS spike mediated entry relative to XBB.1.5,RBD yeast-display DMS ACE2 affinity relative to XBB.1.5,RBD yeast-display DMS RBD expression relative to XBB.1.5,RBD yeast-display DMS escape relative to XBB.1.5,EVEscape relative to XBB.1.5,Hamming distance relative to XBB.1.5
596,BA.1,2021-12-07,B.1.1,A67V T95I Y145D L212I G339D S371L S373P S375F ...,36,I19T -24L -25P -26P S27A A67V H69- V70- A83V T...,False,False,False,NaN,NaN,-3.254,-0.6102,-1.922,-0.94,-0.02,0.4537,47.07,35
597,BA.1.1,2022-01-07,BA.1,A67V T95I Y145D L212I G339D R346K S371L S373P ...,37,I19T -24L -25P -26P S27A A67V H69- V70- A83V T...,False,False,False,NaN,NaN,-3.079,-0.6185,-1.774,-0.81,0.22,0.4537,46.93,35
598,BA.1.1.1,2022-02-23,BA.1.1,A67V T95I Y145D L212I G339D R346K S371L S373P ...,37,I19T -24L -25P -26P S27A A67V H69- V70- A83V T...,False,False,False,NaN,NaN,-3.079,-0.6185,-1.774,-0.81,0.22,0.4537,46.93,35
601,BA.1.1.2,2022-02-23,BA.1.1,A67V T95I Y145D L212I G339D R346K S371L S373P ...,37,I19T -24L -25P -26P S27A A67V H69- V70- A83V T...,False,False,False,NaN,NaN,-3.079,-0.6185,-1.774,-0.81,0.22,0.4537,46.93,35
602,BA.1.1.3,2022-02-23,BA.1.1,A67V T95I Y145D L212I G339D R346K S371L S373P ...,38,I19T -24L -25P -26P S27A A67V H69- V70- A83V T...,False,False,False,NaN,NaN,-3.193,-0.3278,-1.716,-0.81,0.22,0.4537,49.41,36
